In [1]:
import cv2
import tensorflow as tf
import numpy as np
import os
import glob
import csv


In [2]:
batch_size = 32
img_size = (140,140)
num_classes = 3 #either empty, occupied or not occupied

In [3]:
def prepare(img):
    rescale = tf.keras.layers.experimental.preprocessing.Rescaling(scale=1./255, input_shape=(img_size[0],img_size[1],3))
    img = rescale(img)
    return img

In [4]:
model = tf.keras.models.load_model("my_model")

In [5]:
img_path_list = glob.glob("test.png")
img_path_list

['test.png']

In [6]:
for img_path in  img_path_list:

    #read the image
    image = cv2.imread(img_path)
    csv_path = 'camera.csv'

    with open(csv_path, newline='') as f:
        boxes = list(csv.reader(f))
    
    #crop images
    image_input = np.zeros(shape=(len(boxes[1:]), img_size[0], img_size[1], 3))
    for i, one_box in enumerate(boxes[1:]):
        one_box_x = int(int(one_box[1]))
        one_box_y = int(int(one_box[2]))
        one_box_w = int(int(one_box[3]))
        one_box_h = int(int(one_box[4]))
        img_resized = cv2.resize(image[one_box_y:one_box_y+one_box_h, one_box_x:one_box_x+one_box_w], img_size,
                                 interpolation=cv2.INTER_CUBIC)
        image_input[i] = (prepare(np.expand_dims(img_resized, axis=0)))

    #predict the labels
    output = model.predict(image_input)

    #get the label
    label = np.argmax(output, axis=1)

    # Initialize lists to store IDs
    correct_parking_ids = []
    empty_parking_ids = []
    wrong_parking_ids = []

    colors = [(0, 255, 0), (255, 0, 0), (0, 0, 255)]  # Green, Blue, Red

    for i, one_box in enumerate(boxes[1:]):
        one_box_x = int(int(one_box[1]))
        one_box_y = int(int(one_box[2]))
        one_box_w = int(int(one_box[3]))
        one_box_h = int(int(one_box[4]))

        color = colors[label[i]]

        image = cv2.rectangle(image, (one_box_x, one_box_y), (one_box_x + one_box_w, one_box_y + one_box_h),
                          color, 4)
        cv2.putText(image, ('ID:' + str(one_box[0])), (one_box_x, one_box_y - 10), cv2.FONT_HERSHEY_SIMPLEX, 1,
                color, 2)
        # Append the ID to the appropriate list
        if label[i] == 0:
            correct_parking_ids.append(one_box[0])
        elif label[i] == 1:
            empty_parking_ids.append(one_box[0])
        else:
            wrong_parking_ids.append(one_box[0])

    # Print counts and IDs
    print(f"Number of correct parking: {len(correct_parking_ids)}, IDs: {correct_parking_ids}")
    print(f"Number of empty parking: {len(empty_parking_ids)}, IDs: {empty_parking_ids}")
    print(f"Number of wrong parking: {len(wrong_parking_ids)}, IDs: {wrong_parking_ids}")

    #show prediction
    cv2.imshow('prediction', image)
    cv2.waitKey(0)


1/1 [==============================] - 10s 10s/step
Number of correct parking: 4, IDs: ['2', '3', '13', '14']
Number of empty parking: 4, IDs: ['1', '6', '9', '11']
Number of wrong parking: 6, IDs: ['4', '5', '7', '8', '10', '12']


In [7]:
output,label

(array([[1.4703746e-06, 9.9973422e-01, 2.6430955e-04],
        [9.9751544e-01, 7.1082616e-07, 2.4838706e-03],
        [9.9933666e-01, 7.8070525e-07, 6.6252856e-04],
        [6.6238290e-06, 1.8977570e-06, 9.9999154e-01],
        [4.6834390e-04, 7.6663237e-06, 9.9952400e-01],
        [1.2930115e-06, 9.9986649e-01, 1.3216701e-04],
        [8.9241405e-07, 9.0071936e-08, 9.9999905e-01],
        [8.2640567e-05, 2.4582200e-06, 9.9991488e-01],
        [2.5331555e-04, 7.5245386e-01, 2.4729283e-01],
        [1.2627353e-04, 1.2874258e-06, 9.9987245e-01],
        [2.8097745e-06, 9.9985838e-01, 1.3877937e-04],
        [1.5433191e-03, 6.8494719e-06, 9.9844980e-01],
        [9.9773765e-01, 7.0540891e-06, 2.2552891e-03],
        [9.9980456e-01, 1.1633880e-06, 1.9421735e-04]], dtype=float32),
 array([1, 0, 0, 2, 2, 1, 2, 2, 1, 2, 1, 2, 0, 0], dtype=int64))